<a href="https://colab.research.google.com/github/sthagstotz/colab_support/blob/master/P_mg_cross_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import requests

import numpy as np
import pylab as plt


Calculate the scale-dependent correlation coefficient between matter and galaxy power spectra

$$ r(k)= \frac{P_{mg}}{\sqrt{P_{gg} P_{mm}}}$$

where we assume that the galaxy power spectrum can be written as

$$ P_{gg} = b_a^2 P_{mm} + P_\mathrm{shot} $$

with a constant shot noise term. The following interactive plots calculate the correlation coefficient $r$ for chosen values of the (large-scale) bias $b_a$ and the shot noise.

In [0]:
# first, download power spectrum data
url = 'https://raw.githubusercontent.com/sthagstotz/colab_support/master/boss_matterpower.dat'
r = requests.get(url, allow_redirects=True)
open('boss_matterpower.dat', 'wb').write(r.content)

15718

In [14]:
# Interactive plot showing the correlation coefficient

shotnoise = 3500.
bias = 2.

data = np.loadtxt('boss_matterpower.dat')
karray = data[:,0]
pk = data[:,1]

@interact
def set_shot_scale(
    a_bias=widgets.FloatSlider(min=0.,max=5.,step=0.1,value=2.,description='bias'),
    shotnoise=widgets.IntSlider(min=100.,max=10000.,step=100,value=3500.,description='shot noise')
    ):

    #fiducial values: shotnoise = 3500, bias = 2
    P_mm = pk
    #fiducial values
    a_bias_fid, P_shot_fid = 2., 3500.
    P_gg = (a_bias_fid)**2 * pk + P_shot_fid
    r_fid = a_bias_fid * pk / np.sqrt(P_mm * P_gg)
    plt.semilogx(karray, r_fid, 'C0', alpha=0.5)

    P_mm = pk
    P_gg = a_bias**2 * pk + shotnoise
    r = a_bias * pk / np.sqrt(P_mm * P_gg)
    plt.semilogx(karray, r, 'C0')

    plt.xlim(1e-2)
    plt.ylim(0.2, 1.0)
    plt.xlabel('$k$ [Mpc/h]')
    plt.ylabel('correlation coefficient $r$')

interactive(children=(FloatSlider(value=2.0, description='bias', max=5.0), IntSlider(value=3500, description='…

Check for the additional effect of redshift space distrotions (RSD) on the correlation coefficient, using
$$ P_{gg}^{\rm RSD} = b_a^2 \left(1 + \frac{2}{3} \beta + \frac{1}{5} \beta^2 \right) P_{mm} + P_{\rm shot}$$
with
$$ \beta = \frac{\Omega_m(z)^{0.545}}{b_a}$$

In [15]:
@interact
def set_shot_scale(
    a_bias=widgets.FloatSlider(min=0.,max=5.,step=0.1,value=2.,description='bias'),
    shotnoise=widgets.IntSlider(min=100.,max=10000.,step=100,value=3500.,description='shot noise')
    ):
    #fiducial values: shotnoise = 3500, bias = 2
    P_mm = pk
    #fiducial values
    a_bias_fid, P_shot_fid = 2., 3500.
    P_gg = (a_bias_fid)**2 * pk + P_shot_fid
    r_fid = a_bias_fid * pk / np.sqrt(P_mm * P_gg)
    plt.semilogx(karray, r_fid, 'C0', alpha=0.5)

    P_mm = pk
    P_gg = a_bias**2 * pk + shotnoise
    r = a_bias * pk / np.sqrt(P_mm * P_gg)
    plt.semilogx(karray, r, 'C0', label='shot noise contribution')

    #including RSD
    beta = 0.78 / a_bias_fid #Omega_m(z=0.57)**0.545 ~ 0.78 for Planck
    rsd = (1. + 2./3 * beta + 1./5 * beta**2)
    P_rsd = a_bias_fid**2 * rsd * pk + P_shot_fid
    r_rsd_fid = a_bias_fid * pk / np.sqrt(P_mm * P_rsd)
    plt.semilogx(karray, r_rsd_fid, 'C1', alpha=0.5)

    beta = 0.78 / a_bias
    rsd = (1. + 2./3 * beta + 1./5 * beta**2)
    P_rsd = a_bias**2 * rsd * pk + shotnoise
    r_rsd = a_bias * pk / np.sqrt(P_mm * P_rsd)
    plt.semilogx(karray, r_rsd, 'C1', label='including RSD')

    plt.xlim(1e-2)
    plt.ylim(0.2, 1.0)
    plt.xlabel('$k$ [Mpc/h]')
    plt.ylabel('correlation coefficient $r$')
    plt.legend(frameon=False)

interactive(children=(FloatSlider(value=2.0, description='bias', max=5.0), IntSlider(value=3500, description='…